In [1]:
import pandas as pd
import altair as alt
alt.renderers.enable('altair_viewer')
alt.data_transformers.disable_max_rows()
import KDE_prepare_data
import KDE_utils

## MP particle size correlations

*This calculates probability density distributions for MP sizes. These are used to estimate the abundances of MP of specific size ranges, wich can then be investigated for correlations to environmental paramteres such as TOC or sediment grain size properties.*
**Before running, check the computational parameters in KDE_settings.py**

In [2]:
pdd_MP = pd.read_csv('../csv/env_MP_clean_list_SchleiSediments.csv', index_col=0)


In [3]:
sdd_MP = KDE_prepare_data.aggregate_SDD(pdd_MP)
sdd_MP_sed = KDE_prepare_data.add_sediment(sdd_MP)
pdd_sdd_MP = KDE_prepare_data.sdd2pdd(sdd_MP_sed, pdd_MP)

In [4]:
size_pdfs = KDE_utils.per_sample_kde(pdd_sdd_MP)

Schlei_S10:    bandwidth is 50
Schlei_S10_15cm:    bandwidth is 50
Schlei_S11:    bandwidth is 50
Schlei_S13:    bandwidth is 50
Schlei_S14:    bandwidth is 50
Schlei_S15:    bandwidth is 50
Schlei_S17:    bandwidth is 50
Schlei_S19:    bandwidth is 50
Schlei_S1_15cm:    bandwidth is 50
Schlei_S2:    bandwidth is 50
Schlei_S22:    bandwidth is 50
Schlei_S23:    bandwidth is 50
Schlei_S24:    bandwidth is 50
Schlei_S25:    bandwidth is 50
Schlei_S26:    bandwidth is 50
Schlei_S27:    bandwidth is 50
Schlei_S3:    bandwidth is 50
Schlei_S30:    bandwidth is 50
Schlei_S31:    bandwidth is 50
Schlei_S5:    bandwidth is 50
Schlei_S8:    bandwidth is 50


In [5]:
alt.Chart(size_pdfs.melt(id_vars=['x_d'])).mark_line().encode(
    x='x_d',
    y='value',
    color='variable',
    tooltip='variable'
).interactive()

Displaying chart at http://localhost:23590/

In [5]:
def optimise(df):
    df_r = pd.DataFrame(columns=['alpha', 'beta',
                                 'r_BDI', 'p_BDI',
                                 'r_IQI', 'p_IQI'])

    p = np.arange(Config.minParameterForBDI, Config.maxParameterForBDI, Config.stepParameterForBDI)
    for alpha in p:
        for beta in p:
            if alpha == 0 and beta ==0:
                pass
            else:
                df = make_BDI(df, alpha, beta)
                r_BDI = stats.pearsonr(df.particle_loss, df.BDI)
                r_IQI = stats.pearsonr(df.particle_loss, df.IQI)
                df_r.loc[len(df_r)] = [alpha, beta,
                                       r_BDI[0], r_BDI[1],
                                       r_IQI[0], r_IQI[1]]
                print(f'Running BDI optimisation with alpha =     {round(alpha, 2)}                        ', end="\r", flush=True)

    # print(df_r.loc[df_r.r_IQI == df_r.r_IQI.max()])
    print(df_r.loc[df_r.r_BDI == df_r.r_BDI.max()])
    bestAlpha, bestBeta = df_r.loc[df_r.r_BDI == df_r.r_BDI.max()].iloc[0, 0:2]

    return bestAlpha, bestBeta
